# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`CHEM-Data`*

---
*@author: Evan*\
*@date: 2023-05-17*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from functools import reduce

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.io.shapereader import Reader

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
isam = xr.open_dataset('F:/Data/Project_anqing/April/add_data/COMBINE_ACONC_CN3AH_135X138_2023_isam.nc')
grid = xr.open_dataset('F:/Data/Project_anqing/GRIDCRO2D_2023021.nc')

In [3]:
times=pd.date_range('2023-04-06-00','2023-04-10-23',freq='h')
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50

In [6]:
days=1 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from ISAM
        O3_AQ=(['time','level','y','x'],isam.O3_AQ[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing','units':'ug m-3'}),
        O3_HF=(['time','level','y','x'],isam.O3_HF[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Hefei','units':'ug m-3'}),
        O3_TL=(['time','level','y','x'],isam.O3_TL[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Tongling','units':'ug m-3'}),
        O3_CZ=(['time','level','y','x'],isam.O3_CZ[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Chizhou','units':'ug m-3'}),
        O3_JJ=(['time','level','y','x'],isam.O3_JJ[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Jiujiang','units':'ug m-3'}),
        O3_OTH=(['time','level','y','x'],isam.O3_OTH[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from OTHER','units':'ug m-3'}),
        O3_ICO=(['time','level','y','x'],isam.O3_ICO[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from ICON','units':'ug m-3'}),
        O3_BCO=(['time','level','y','x'],isam.O3_BCO[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from BCON','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres[:26],
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202304',
        grid='CN3AH_135X138',
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 120, level: 26, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-04-06 ... 2023-04-10T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 703.9 659.0 616.2
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    O3_AQ      (time, level, y, x) float32 0.0006611 0.007224 ... 1.066e-07
    O3_HF      (time, level, y, x) float32 0.0 0.0 0.0 ... 0.09503 0.1023
    O3_TL      (time, level, y, x) float32 1.142e-14 3.15e-13 ... 1.156e-06
    O3_CZ      (time, level, y, x) float32 4.012e-11 9.542e-10 ... 7.834e-07
    O3_JJ      (time, level, y, x) float32 0.4769 1.427 ... 4.054e-13 3.229e-13
    O3_OTH     (time, level, y, x) float32 28.8 28.75 28.36 ... 0.1064 0.1147
    O3_ICO     (time, level, y, x) float32 29.26 29.83 ... 4.168e-06 4.419e-06
    O3_BCO     (time, level, y, x) float32 5.116 4.79 4.117 ... 84.31 84.31
Attributes:
    case:     Anqing_202304
    grid:     CN3AH_135X138

In [7]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/April_05wind_ISAM.nc',encoding=encoding)

In [8]:
days=1 # set spin-up days
dataset2=xr.Dataset(
    data_vars=dict(
        # ! vars from ISAM
        PM25_AQ=(['time','level','y','x'],isam.PM25_IONS_AQ[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Anqing','units':'ug m-3'}),
        PM25_HF=(['time','level','y','x'],isam.PM25_IONS_HF[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Hefei','units':'ug m-3'}),
        PM25_TL=(['time','level','y','x'],isam.PM25_IONS_TL[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Tongling','units':'ug m-3'}),
        PM25_CZ=(['time','level','y','x'],isam.PM25_IONS_CZ[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Chizhou','units':'ug m-3'}),
        PM25_JJ=(['time','level','y','x'],isam.PM25_IONS_JJ[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Jiujiang','units':'ug m-3'}),
        PM25_OTH=(['time','level','y','x'],isam.PM25_IONS_OTH[days*24-8:-8,:,:,:].data,{'long name':'PM25 from OTHER','units':'ug m-3'}),
        PM25_ICO=(['time','level','y','x'],isam.PM25_IONS_ICO[days*24-8:-8,:,:,:].data,{'long name':'PM25 from ICON','units':'ug m-3'}),
        PM25_BCO=(['time','level','y','x'],isam.PM25_IONS_BCO[days*24-8:-8,:,:,:].data,{'long name':'PM25 from BCON','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres[:26],
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202304',
        grid='CN3AH_135X138',
    ),
)
dataset2

<xarray.Dataset>
Dimensions:    (time: 720, level: 26, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-04-01 ... 2023-04-30T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 703.9 659.0 616.2
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    PM25_AQ    (time, level, y, x) float32 0.01286 0.03075 0.0489 ... 0.0 0.0
    PM25_HF    (time, level, y, x) float32 0.007352 0.01758 0.02803 ... 0.0 0.0
    PM25_TL    (time, level, y, x) float32 0.004421 0.01061 0.01678 ... 0.0 0.0
    PM25_CZ    (time, level, y, x) float32 0.00521 0.01247 0.0198 ... 0.0 0.0
    PM25_JJ    (time, level, y, x) float32 0.1628 0.09616 0.109 ... 0.0 0.0 0.0
    PM25_OTH   (time, level, y, x) float32 0.03686 0.08821 0.1406 ... 0.0 0.0
    PM25_ICO   (time, level, y, x) float32 1.099e-13 2.623e-13 ... 0.0 0.0
    PM25_BCO   (time, level, y, x) float32 5.515 6.288 7.302 ... 0.7358 0.7371
Attributes:
    case:     Anqing_202304
    grid:     CN3AH_135X138

In [9]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset2.data_vars}
dataset2.to_netcdf('D:/Download/April_ISAM2.nc',encoding=encoding)